In [ ]:
# move into once tableN_docker models notebook runs (this code does a sanity check with my work)

import pandas as pd
import yaml

extra_options = {
    'CF':['All', 'OVERALL'],
    'AKLIMATE':[ 'MULTI', 'TOP'], 
    'skgrid':[ 'OVERALL'],
    'subSCOPE':[ 'OVERALL'],
    'jadbio':['MULTI']

}

nameconvert = {
    'aklimate':'AKLIMATE', 
    'cloudforest':'CF',
    'jadbio':'jadbio',
    'skgrid':'skgrid',
    'subscope':'subSCOPE'
}


with open('../data/table_docker_info/options_extended.yml', 'r') as fh2:
    best = yaml.load(fh2, Loader=yaml.Loader)
# sanity check my filtering matches christina's
# read in christina data 
perf_df =pd.read_csv('../src/BestModelPerDataTypePerGroup_deduplicated_2022_06_16_fixed_fromChristina.txt', sep='\t')


for algor in extra_options.keys():
    for cancer in list(best.keys()):
    #     print(cancer)
        check= {}
        for k,v in best[cancer][algor].items():
            if 'info' in k:
                # aka don't count the nans
                if type(v['Mean_overall_weighted_f1'])!=float:
                    # ignore the TOP and OVERALL (will calculate this later)
                    if k.split('_')[1] != 'TOP' and k.split('_')[1]!='OVERALL':
                        check[k.split('_')[1]]=v['Mean_overall_weighted_f1']

        for check_plat in check.keys():
            # alimate, jabio reports MULTI as ALL in christinas table
            # CF reports All as ALL in christinas table
            if check_plat == 'MULTI' or check_plat == 'All': 
                print('MULTI for ', algor)
                s1 = perf_df[(perf_df['cohort']==cancer)&(perf_df['feature_list_method']==algor)][['featureID','cohort','performance_metric','Mean','Std','feature_list_method', 'datatype']]
                perf_df_Mean = list(s1[s1['datatype']=='ALL']['Mean'])[0]
                assert perf_df_Mean == check[check_plat], 'issue with not matchingup for {} perfdf mean and {} christinas mean. {} {} {}'.format(perf_df_Mean, check[check_plat], cancer, algor, check_plat)        
            else:
    #             print('checking')
                s1 = perf_df[(perf_df['cohort']==cancer)&(perf_df['feature_list_method']==algor)][['featureID','cohort','performance_metric','Mean','Std','feature_list_method', 'datatype']]
                perf_df_Mean = list(s1[s1['datatype']==check_plat]['Mean'])[0]
                assert perf_df_Mean == check[check_plat], 'issue with not matchingup for {} perfdf mean and {} christinas mean. {} {} {}'.format(perf_df_Mean, check[check_plat],  cancer, algor, check_plat)

### sanity check

AssertionError: issue with not matchingup for 0.529 perfdf mean and 0.553 christinas mean. ACC AKLIMATE CNVR

In [ ]:
bmatrix_raw= pd.read_csv('../src/classifier_metrics_20220511/big_results_matrix.tsv', sep='\t', low_memory=False)

#####
grp = 'AKLIMATE'
c = 'ACC'
p = 'CNVR'
#####
bmatrix = bmatrix_raw[bmatrix_raw['feature_list_method'] == grp]
bmatrix = bmatrix[bmatrix['cohort'] == c]
bmatrix = bmatrix[bmatrix['performance_metric'] == pmetric].reset_index(drop=True)
bmatrix = bmatrix.sort_values(by='Mean', ascending=False)
ms = [a for a in bmatrix['model'] if p in a]
bmatrix[bmatrix['model'].isin(ms)]

AssertionError: issue with not matchingup for 0.529 perfdf mean and 0.553 christinas mean. ACC AKLIMATE CNVR

In [ ]:
bmatrix_raw[bmatrix_raw['total_features']!= '__NO_LIST__'].reset_index(drop=True).shape


In [ ]:
bmatrix_raw.shape

### end of sanity check

### continue with adding in the OVERALL category for that algor

In [ ]:
with open('../data/table_docker_info/options_extended.yml', 'r') as fh2:
    best = yaml.load(fh2, Loader=yaml.Loader)

# determine which algors need to calculate overall
models_need_overall= []
for k,v in extra_options.items():
    if 'OVERALL' not in v:
        models_need_overall.append(k)
# add in OVERALL = highest performance of all models from that algor
# thus will say which model of AKLIAMTE is OVERALL, then for skgrid, etc
for algor in models_need_overall:
    for cancer in list(best.keys()):
        # find the model 
        overall_4_algor = {'model':'NA', 'score': 0}
        for KEY in [a for a in best[cancer][algor].keys() if 'info' in a]:
            perform = best[cancer][algor][KEY]['Mean_overall_weighted_f1']
            if perform > overall_4_algor['score']:
                overall_4_algor['score']=perform
                overall_4_algor['model']=KEY
        best[cancer][algor]['OVERALL']= best[cancer][algor][KEY.split('_')[1]]
        best[cancer][algor]['info_OVERALL']= best[cancer][algor][overall_4_algor['model']]


In [ ]:
best

In [ ]:
# pull up info used in docker models
with open('../../../09_docker/gdan-tmp-models/tools/model_info.json', 'r') as file:
    info = yaml.safe_load(file)

reverse_name = {
    'AKLIMATE':'aklimate', 
    'CF':'cloudforest',
    'jadbio':'jadbio',
    'skgrid':'skgrid',
    'subSCOPE':'subscope'
}


In [ ]:
best[cancer][algor][platform] # NO_MODEL

In [ ]:
# combine all info into 1 df
# #####
algor = 'CF'
cancer = 'BRCA'
# #####
# skgrid

res = {'Command':[], 'Mean_Overall_Weighted_F1':[],'Std_overall_weighted_f1':[], 'Cohort': [], 'Platform': [], 'Algorithm_Method':[],'Features':[]}
for cancer in best.keys():
    for algor in best[cancer].keys():
        
        platform_cyles = [a for a in best[cancer][algor] if 'info' in a]
        # update the above
        
        for pc in platform_cyles:
            platform = pc.split('_')[1]
            # skip when don't have model info on that 
            if 'NO_MODEL' not in best[cancer][algor][platform]:
                res['Command'].append('bash RUN_model.sh {} {} {} YOUR-DATA.tsv'.format(cancer, platform, reverse_name[algor]))

                mean_value = best[cancer][algor][pc]['Mean_overall_weighted_f1']
                res['Mean_Overall_Weighted_F1'].append(mean_value)

                sd_value = best[cancer][algor][pc]['Std_overall_weighted_f1']
                res['Std_overall_weighted_f1'].append(sd_value)

                res['Cohort'].append(cancer)

                # modify in table TOP to OVERALL but command col will use TOP still
                if platform == 'TOP':
                    res['Platform'].append('OVERALL')
                elif platform == 'All':
                    res['Platform'].append('MULTI')
                else:
                    res['Platform'].append(platform)

                res['Algorithm_Method'].append(reverse_name[algor])

                fts = info[reverse_name[algor]][cancer][platform]['fts']
                res['Features'].append(fts)
            else:
                res['Command'].append('NA')

                res['Mean_Overall_Weighted_F1'].append('NA')

                res['Std_overall_weighted_f1'].append('NA')

                res['Cohort'].append(cancer)

                # modify in table TOP to OVERALL but command col will use TOP still
                if platform == 'TOP':
                    res['Platform'].append('OVERALL')
                elif platform == 'All':
                    res['Platform'].append('MULTI')
                else:
                    res['Platform'].append(platform)

                res['Algorithm_Method'].append(reverse_name[algor])

                res['Features'].append('NA')                



In [ ]:
info[reverse_name[algor]][cancer].keys()
algor

In [ ]:
info[reverse_name[algor]][cancer].keys()



In [ ]:
summary = pd.DataFrame.from_dict(res)
summary = summary.sort_values(by = ['Mean_Overall_Weighted_F1','Cohort', 'Platform','Algorithm_Method'], ascending =[False, False,False, False]).reset_index(drop=True)

In [ ]:
summary

In [ ]:
summary.sort_values(by = 'Cohort', ascending =True).reset_index(drop=True)